In [1]:
MESSAGE_SYSTEM_CONTENT = """You are a customer service agent that helps a customer with answering questions. 
Please answer the question based on the provided context below. 
Make sure not to make any changes to the context, if possible, when preparing answers to provide accurate responses. 
If the answer cannot be found in context, just politely say that you do not know, do not try to make up an answer."""

'nt'

In [ ]:
DOCUMENT:
(document text)

QUESTION:
(users question)

INSTRUCTIONS:
Answer the users QUESTION using the DOCUMENT text above.
Keep your answer ground in the facts of the DOCUMENT.
If the DOCUMENT doesn’t contain the facts to answer the QUESTION return {NONE}

In [ ]:
the document contains field that need to be filled by users. Collect these fields and their value and organize them into a logical grouping.

The ouput needs to be in a python dictionary format with  field name go into keys and values go into items.iter

Example:

Employer goes to Key
John goes to item

{employer:'John', ingredients}

In [ ]:
The document is a recipe. You are to retrieve the name of the dish, ingredients, method of cooking. Also, you need to rate how much oily is this recipe. Criteria is 0 means no oil involved and 5 means it was deep fried.

Out the information in a json format. 

Example:
{dish_name: 'steak', 
 ingredients: "1 egg, 500 ml water, 2 cups flour",
 method: "prepare and put into oven",
 oil_rating: 2}

In [ ]:
import streamlit as st

from langchain_community.document_loaders import SeleniumURLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_ollama import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""

embeddings = OllamaEmbeddings(model="llama3.2")
vector_store = InMemoryVectorStore(embeddings)

model = OllamaLLM(model="llama3.2")

def load_page(url):
    loader = SeleniumURLLoader(
        urls=[url]
    )
    documents = loader.load()

    return documents

def split_text(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        add_start_index=True
    )
    data = text_splitter.split_documents(documents)

    return data

def index_docs(documents):
    vector_store.add_documents(documents)

def retrieve_docs(query):
    return vector_store.similarity_search(query)

def answer_question(question, context):
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model
    return chain.invoke({"question": question, "context": context})

st.title("AI Crawler")
url = st.text_input("Enter URL:")

documents = load_page(url)
chunked_documents = split_text(documents)

index_docs(chunked_documents)

question = st.chat_input()

if question:
    st.chat_message("user").write(question)
    retrieve_documents = retrieve_docs(question)
    context = "\n\n".join([doc.page_content for doc in retrieve_documents])
    answer = answer_question(question, context)
    st.chat_message("assistant").write(answer)

In [8]:
data = "1 egg, 500 ml water, 2 cups flour"
print(data.replace(", ","\n"))

1 egg
500 ml water
2 cups flour
